In [1]:
### 01 INIT

import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import time

load_dotenv()
db = create_engine(os.getenv('DATABASE_URL'))

In [2]:
### 02 Check DB-connection, count rows

sql_test = "SELECT COUNT(*) FROM emails;"
df_test = pd.read_sql(sql_test, db)
print(df_test)


    count
0  517401


In [3]:
### 03 Add noise_type column, prepare noise-filtering

with db.connect() as conn:
    conn.execute(text("ALTER TABLE emails ADD COLUMN IF NOT EXISTS noise_type TEXT;"))
    conn.execute(text("ALTER TABLE emails ADD COLUMN IF NOT EXISTS sender_clean TEXT;"))
    conn.execute(text("ALTER TABLE emails ADD COLUMN IF NOT EXISTS recipients_clean TEXT[];"))
    conn.commit()
print("Columns 'noise_type', 'sender_clean', 'recipients_clean' created.")

Columns 'noise_type', 'sender_clean', 'recipients_clean' created.


In [4]:
### 04 Clean email addresses

sql_clean_addresses = """
UPDATE emails SET
    sender_clean = LOWER(TRIM(
        COALESCE(substring(sender from '<([^>]+)>'), sender)
    )),
    recipients_clean = (
        SELECT array_agg(DISTINCT cleaned_recipient)
        FROM (
            SELECT LOWER(TRIM(
                COALESCE(substring(raw_recipient from '<([^>]+)>'), raw_recipient)
            )) AS cleaned_recipient
            FROM unnest(string_to_array(regexp_replace(recipients, E'[;\\r\\n\\t]+', ',', 'g'), ',')) AS raw_recipient
        ) AS sub
        WHERE cleaned_recipient LIKE '%@%'
    );
"""
with db.connect() as conn:
    conn.execute(text(sql_clean_addresses))
    conn.commit()
print("Address cleaning complete.")

Address cleaning complete.


In [5]:
### 05 Identify most frequent subjects

sql_top_subjects = """
SELECT 
    subject, 
    COUNT(*) as frequency
FROM emails
GROUP BY subject
ORDER BY frequency DESC
LIMIT 100;
"""

print("Querying the most frequent subjects...")
df_subjects = pd.read_sql(sql_top_subjects, db)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
display(df_subjects)

Querying the most frequent subjects...


,subject,frequency
0,None,19187
1,RE:,6477
2,Re:,6306
3,Demand Ken Lay Donate Proceeds from Enron Stock Sales,1124
4,FW:,938
5,Schedule Crawler: HourAhead Failure,900
6,Schedule Crawler: HourAhead Failure <CODESITE>,800
7,Enron Mentions,784
8,EnTouch Newsletter,518
9,Organizational Announcement,378


In [6]:
### 06 Identify most frequent body text

sql_top_body = """
SELECT 
    body_text, 
    COUNT(*) as frequency
FROM emails
GROUP BY body_text
ORDER BY frequency DESC
LIMIT 100;
"""

print("Querying the most frequent body text...")
df_body = pd.read_sql(sql_top_body, db)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
display(df_body)

Querying the most frequent body text...


,body_text,frequency
0,"As you know, Enron Net Works (ENW) and Enron Global Strategic Sourcing (GSS) recently executed a two-year agreement, whereby MCI WorldCom would serve as Enron's primary telecommunications provider. In our previous communication, we indicated that we would provide you with more detailed information as it became available.\r\n\r\nBeginning Friday, October 26, 2001 at 9 a.m. C.S.T, the procedure for calling Enron's Houston offices from international locations, excluding Canada, using the 800-97-Enron phone number will change. The new procedure is as follows: \r\n\r\n1.\tPlease dial the WorldPhone International Access number for the country where you are located (country access code), which is available on the attached wallet card, accessible through the following link: http://home.enron.com:84/messaging/mciannouncement.doc. \r\n2.\tYou will then be prompted for your PIN number. Since calling cards and pin numbers are not required to use this service, all users should respond by dialing 1-800-97-ENRON or 1-800-973-6766. \r\n3.\tYou will then be asked to enter your destination. Please dial 0-800-97-Enron (800-973-6766) to reach Enron's corporate offices in Houston.\r\n \r\nThis procedure can only be used to call 800-97-Enron from WorldPhone International locations. If you are calling from the U. S. or Canada, please continue to dial 1-800-97-ENRON.\r\n \r\nIf you have questions regarding commercial aspects of this agreement, please feel free to contact Tom Moore, GSS senior contract manager at 713-345-5552. For technical issues, please contact Hasan Imam, ENW IT manager at 713-345-8525. \r\n",112
1,"We've updated the Merger Q&A document on our Enron Updates site ( <http://home.enron.com/updates/mergerQA.html>), as a result of the many questions you've had concerning the merger between Enron and Dynegy. Questions addressed include those about Enron stock options, benefits and immigration status. Please stay tuned for additional updates.",110
2,"Ken Lay and Jeff Skilling were interviewed on CNNfn to discuss the succession \r\nof Jeff to CEO of Enron. We have put the interview on IPTV for your viewing \r\npleasure. Simply point your web browser to http://iptv.enron.com, click the \r\nlink for special events, and then choose ""Enron's Succession Plan."" The \r\ninterview will be available every 15 minutes through Friday, Dec. 15.",107
3,"\r\nOur natural gas business continues to benefit from effective account management and resource allocation focused on identifying and responding to the needs of our varied customers. In order to keep our organization optimally structured and to facilitate additional growth, we are making the following changes:\r\n\r\nProducer/Wellhead Group \r\nThe current mid-market, origination and wellhead pricing activity currently within the Central and Eastern Gas Regions will be consolidated with the Derivatives group under Fred Lagrasta. This will create a single business unit focused upon the needs of the producing industry within the Eastern U.S. The producer focus in the Western U.S. and Texas will remain unchanged reporting to Mark Whitt and Brian Redmond respectively.\r\n\r\nStrategic Asset Development\r\nLaura Luce will move from her role in the Central Region to lead an effort focused strictly on identifying and entering into long-term strategic arrangements within the Central and Eastern Regions. This initiative will focus on a limited number of selected markets that provide strategic opportunities for partnering in asset development, asset management and optimization. This effort will continue to work very closely with the regional leads.\r\n\r\nCentral Origination and Mid-Market\r\nFrank Vickers will continue his current role in the Eastern Region and will assume the leadership role for Mid-Market and Origination activity in the Central Region. \r\n \r\n\r\nThere will be no changes to the West and Texas Origination groups headed respectively by Barry Tycholiz and Brian Redmond

In [7]:
### 07 Analyze content duplicates

print("Analyzing content duplicates...")

sql_duplicate_stats = """
WITH content_groups AS (
    SELECT 
        COUNT(*) as group_size
    FROM emails
    WHERE noise_type IS NULL
    GROUP BY sender_clean, sent_at_utc, recipients_clean, body_text
)
SELECT
    (SELECT COUNT(*) FROM emails WHERE noise_type IS NULL) as total_relevant_mails,
    COUNT(*) as unique_content_mails,
    (SELECT COUNT(*) FROM emails WHERE noise_type IS NULL) - COUNT(*) as duplicate_rows
FROM content_groups;
"""
df_stats = pd.read_sql(text(sql_duplicate_stats), db)

print("Duplicate statistics")
display(df_stats)

sql_find_duplicates_rich = """
WITH duplicate_groups AS (
    SELECT
        sender_clean,
        sent_at_utc,
        recipients_clean,
        body_text,
        COUNT(*) as duplicate_count,
        MIN(id) as example_id
    FROM emails
    WHERE noise_type IS NULL
    GROUP BY sender_clean, sent_at_utc, recipients_clean, body_text
    HAVING COUNT(*) > 1
)
SELECT 
    dg.duplicate_count,
    e.sender as original_sender,
    dg.sender_clean,
    dg.sent_at_utc,
    SUBSTRING(dg.body_text, 1, 100) as body_preview
FROM duplicate_groups dg
JOIN emails e ON dg.example_id = e.id
ORDER BY dg.duplicate_count DESC
LIMIT 100;
"""

df_examples = pd.read_sql(text(sql_find_duplicates_rich), db)

print("Top 100 duplicate examples (with original data)")
pd.set_option('display.max_colwidth', 80)
display(df_examples)

Analyzing content duplicates...
Duplicate statistics


,total_relevant_mails,unique_content_mails,duplicate_rows
0,517401,255444,261957


Top 100 duplicate examples (with original data)


,duplicate_count,original_sender,sender_clean,sent_at_utc,body_preview
0,107,public.relations@enron.com,public.relations@enron.com,2000-12-13 14:34:00+00:00,Ken Lay and Jeff Skilling were interviewed on CNNfn to discuss the successio...
1,104,no.address@enron.com,no.address@enron.com,2001-10-25 02:22:28+00:00,\r\nOur natural gas business continues to benefit from effective account man...
2,98,no.address@enron.com,no.address@enron.com,2001-11-19 21:34:59+00:00,\r\nI know that this is a difficult time for all of us. With everything goi...
3,98,enron.announcements@enron.com,enron.announcements@enron.com,2000-12-13 06:01:00+00:00,It is my great pleasure to announce that the Board has accepted my \r\nrecom...
4,96,chairman.office@enron.com,chairman.office@enron.com,2001-10-26 23:11:01+00:00,"Trade press, recruiting firms and others recently have made numerous calls t..."
5,96,no.address@enron.com,no.address@enron.com,2001-10-26 04:43:45+00:00,"\r\nIf you are a participant in the Enron Corp. Savings Plan, please read th..."
6,94,no.address@enron.com,no.address@enron.com,2001-10-30 03:06:37+00:00,\r\neSource Presents Lexis-Nexis Training\r\n\r\nBasic\r\n\r\nLexis-Nexis Ba...
7,94,no.address@enron.com,no.address@enron.com,2001-11-21 16:16:53+00:00,"\r\nAs you know, this is an unprecedented time in Enron's history. We must ..."
8,91,no.address@enron.com,no.address@enron.com,2001-10-22 11:55:34+00:00,"The All-Employee Meeting will be held Tuesday, Oct. 23, at 10 a.m. Houston t..."
9,91,no.address@enron.com,no.address@enron.com,2001-10-23 04:00:26+00:00,Attached is a new link for employees unable to attend the all-employee meeti...


In [8]:
### 08 Mark noise by LLM-created keyword list based on previous results.

sql_reset_noise_column = "UPDATE emails SET noise_type = NULL;"

with db.connect() as conn:
    conn.execute(text(sql_reset_noise_column))
    conn.commit()

sql_mark_pattern_based_noise = """
UPDATE emails 
SET noise_type = 'NOISE'
WHERE 
    noise_type IS NULL AND (

    -- Section 1: Noise identified by frequent Subjects

    -- 1a. System- & IT-Patterns
    subject ILIKE 'Automatic reply:%' OR
    subject ILIKE 'Undeliverable:%' OR
    subject ILIKE 'Out of Office:%' OR
    subject ILIKE 'Schedule Crawler:%' OR
    subject ILIKE 'Synchronization Log:%' OR
    subject ILIKE 'WARNING: Your mailbox%' OR
    subject ILIKE 'Weekend Systems Availability%' OR
    subject ILIKE 'An Inbound Message For You Has Been Quarantined' OR
    subject ILIKE 'Expense Reports Awaiting Your Approval' OR
    subject ILIKE 'URGENT - TO PREVENT LOSS OF INFORMATION'
    
    -- 1b. Newsletters & Newsfeeds
    OR subject ILIKE 'Enron Mentions%' OR
    subject ILIKE 'EnTouch Newsletter%' OR
    subject ILIKE 'Williams Energy News Live%' OR
    subject ILIKE 'RIGZONE Industry News%' OR
    subject ILIKE 'Yahoo! Breaking News%' OR
    subject ILIKE 'Btu Weekly%' OR
    subject ILIKE 'Btu''s Weekly Power Report%' OR
    subject ILIKE 'Ameriflash Newsletter%' OR
    subject ILIKE 'CNN Breaking News%' OR
    subject ILIKE '%execution report'
    
    -- 1c. Internal Mass-Announcements
    OR subject ILIKE 'Organizational Announcement%' OR
    subject ILIKE 'Enron Announcement%' OR
    subject ILIKE 'All-Employee Meeting%' OR
    subject ILIKE 'Jeff McMahon Named CFO' OR
    subject ILIKE 'Holiday Party - Canceled' OR
    body_text ILIKE '%http://home.enron.com/updates/mergerQA.html%' OR
    body_text ILIKE '%http://home.enron.com/message/%' OR
    body_text ILIKE '%http://invitation.enron.com%'

    -- 1d. Spam
    OR subject ILIKE 'Demand Ken Lay Donate Proceeds%' OR

    -- Section 2: Noise identified by duplicate analysis

    -- 2a. High-Volume Broadcast Senders
    LOWER(sender_clean) IN (
        'public.relations@enron.com',
        'chairman.office@enron.com',
        'office.chairman@enron.com',
        'announcements.enron@enron.com',
        'technology.enron@enron.com',
        'clickathome@enron.com',
        '40enron@enron.com',
        'jr..legal@enron.com',
        'officeofthechairman2@enron.com',
        'chairman.ken@enron.com',
        'administration.enron@enron.com',
        'ken.skilling@enron.com',
        'chairman.enron@enron.com',
        'coo.jeff@enron.com',
        'infrastructure.ubsw@enron.com',
        'no.address@enron.com',
        'exchange.administrator@enron.com'
    )
    
    -- 2b. High-Volume Global Recipient Lists
    OR recipients ILIKE '%all.worldwide@enron.com%'
    OR recipients ILIKE '%all.employees@enron.com%'
    OR recipients ILIKE '%all.houston@enron.com%'
    OR recipients ILIKE '%dl-ga-all_%'
);
"""

print("Marking noise records based on patterns...")
with db.connect() as conn:
    result = conn.execute(text(sql_mark_pattern_based_noise))
    conn.commit()

print(f"Update completed. {result.rowcount} Rows marked as 'NOISE'.")

# Verification
df_stats = pd.read_sql("SELECT noise_type, COUNT(*) as count FROM emails GROUP BY noise_type", db)
print("Statistic of marked mails:")
display(df_stats)

Marking noise records based on patterns...
Update completed. 23643 Rows marked as 'NOISE'.
Statistic of marked mails:


,noise_type,count
0,NOISE,23643
1,None,493758


In [9]:
### 09 Create new deduplicated table 'emails_clean'

print("Starting deduplication process...")
start_time = time.time()

total_relevant_count = pd.read_sql("SELECT COUNT(*) FROM emails WHERE noise_type IS NULL", db).iloc[0,0]
print(f"Total relevant emails before deduplication: {total_relevant_count}")

sql_deduplicate_precise = """
DROP TABLE IF EXISTS emails_clean;

CREATE TABLE emails_clean AS
SELECT DISTINCT ON (sender_clean, sent_at_utc, recipients_clean, subject, body_text)
    *
FROM 
    emails
WHERE
    noise_type IS NULL;

ALTER TABLE emails_clean ADD PRIMARY KEY (id);
CREATE INDEX idx_clean_sent_at_utc ON emails_clean (sent_at_utc);
CREATE INDEX idx_clean_sender ON emails_clean (sender_clean);
"""

print("Creating deduplicated table 'emails_clean'...")
with db.connect() as conn:
    conn.execute(text(sql_deduplicate_precise))
    conn.commit()

clean_count = pd.read_sql("SELECT COUNT(*) FROM emails_clean", db).iloc[0,0]
duplicates_removed = total_relevant_count - clean_count

print("-" * 30)
print(f"Process finished in {time.time() - start_time:.2f}s.")
print(f"Unique emails remaining: {clean_count}")
print(f"Duplicates removed: {duplicates_removed}")
print("-" * 30)

Starting deduplication process...
Total relevant emails before deduplication: 493758
Creating deduplicated table 'emails_clean'...
------------------------------
Process finished in 81.56s.
Unique emails remaining: 248940
Duplicates removed: 244818
------------------------------


In [10]:
### 10 Create communication network tables
# 'communication_links': detailed link table (one row per mail per recipient)
# 'communication_network': create aggregated network table (who emailed whom how often?)

MAX_RECIPIENTS = 5 

sql_build_network_tables = f"""
DROP TABLE IF EXISTS communication_links;
CREATE TABLE communication_links AS
SELECT
    id as email_id,
    sent_at_utc,
    sent_at_local,
    sender_clean as sender,
    unnest(recipients_clean) as recipient
FROM
    emails_clean
WHERE
    sender_clean IS NOT NULL 
    AND recipients_clean IS NOT NULL
    AND cardinality(recipients_clean) BETWEEN 1 AND {MAX_RECIPIENTS};

DELETE FROM communication_links 
WHERE recipient IS NULL OR recipient = '' OR recipient NOT LIKE '%@%';

CREATE INDEX idx_links_sender ON communication_links (sender);
CREATE INDEX idx_links_recipient ON communication_links (recipient);


DROP TABLE IF EXISTS communication_network;
CREATE TABLE communication_network AS
SELECT
    sender,
    recipient,
    COUNT(*) as message_count
FROM
    communication_links
GROUP BY
    sender, recipient;

CREATE INDEX idx_network_sender ON communication_network (sender);
CREATE INDEX idx_network_recipient ON communication_network (recipient);
"""

print(f"Building network tables (Filter: Max {MAX_RECIPIENTS} recipients)...")
start_time = time.time()
with db.connect() as conn:
    conn.execute(text(sql_build_network_tables))
    conn.commit()
print(f"Finished in {time.time() - start_time:.2f}s.")

# Check
links_count = pd.read_sql("SELECT COUNT(*) FROM communication_links", db).iloc[0,0]
network_count = pd.read_sql("SELECT COUNT(*) FROM communication_network", db).iloc[0,0]
print(f"Stats: {links_count} individual links created.")
print(f"Stats: {network_count} unique sender-recipient pairs aggregated.")

Building network tables (Filter: Max 5 recipients)...
Finished in 4.31s.
Stats: 275826 individual links created.
Stats: 83678 unique sender-recipient pairs aggregated.
